In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd

In [6]:
#dataのインポート+シャッフル
data = pd.read_csv("./dataset/exp_data.csv",index_col = 0).sample(frac=1, random_state=1).reset_index(drop=True)
print(data.shape)
print(data.columns)

(425, 32)
Index(['class_a', 'class_b', 'relation', 'ans_yamasaki', 'ans_ueda',
       'ans_daisuke', 'ans_hosomi', 'ans_kazuma', 'ans_sugi', 'type', 'title',
       'source', 'sim_by_mcg', 'is_a_by_mcg', 'lexvec_sim', 'wiki2vec_sim',
       'wn_hu_average', 'wn_pu_average', 'wn_hu_best', 'wn_pu_best',
       'wn_sim_path', 'wn_sim_lch', 'wn_sim_wup', 'wn_sim_res', 'wn_sim_jcn',
       'wn_sim_lin', 'has_same_word', 'is_include_word', 'bert_sim',
       'lexvec_diff', 'wiki2vec_diff', 'bert_diff'],
      dtype='object')


In [7]:
#カラム配列定義
#特徴量カラム
feature_columns =  ["sim_by_mcg","is_a_by_mcg","lexvec_sim","wiki2vec_sim","wn_hu_average",
                "wn_pu_average","wn_sim_path","wn_sim_wup",
                "is_include_word","bert_sim","lexvec_diff","wiki2vec_diff","bert_diff"]


#通常の特徴量ラベル
X_normal_columns = ["sim_by_mcg","is_a_by_mcg","lexvec_sim","wiki2vec_sim","wn_hu_average",
                "wn_pu_average","wn_sim_path","wn_sim_wup","is_include_word","bert_sim"]

#ベクトル系特徴量ラベル
pca_di={"lexvec_diff":30,"wiki2vec_diff":30,"bert_diff":3}

#被験者実験データ
ans_columns = [ 'ans_ueda',
       'ans_daisuke', 'ans_kazuma', 'ans_sugi']

In [8]:

# 欠損値は最頻値補完を行う
def fill_na_mean(data):
    data.loc[:,"sim_by_mcg"] = data.loc[:,"sim_by_mcg"].fillna(data.loc[:,"sim_by_mcg"].mode()[0])
    data.loc[:,"is_a_by_mcg"] = data.loc[:,"is_a_by_mcg"].fillna(data.loc[:,"is_a_by_mcg"].mode()[0])
    data.loc[:,"wiki2vec_sim"] = data.loc[:,"wiki2vec_sim"].fillna(data.loc[:,"wiki2vec_sim"].mode()[0])
    data.loc[:,"wiki2vec_diff"] = data.loc[:,"wiki2vec_diff"].fillna(data.loc[:,"wiki2vec_diff"].mode()[0])
    return data
data = fill_na_mean(data)
#クラスラベルエンコーディング
from sklearn.preprocessing import LabelEncoder
def label_encorder(data):
    le = LabelEncoder()
    encoded = le.fit_transform(data['relation'].values)
    for ans_c in ans_columns:
        temp_label = le.transform(data[ans_c].values)
        data[ans_c] = temp_label
    data.loc[:,'relation_label'] = encoded
    print(str(data["relation"].unique())+","+str(le.transform(data["relation"].unique())))
    return data
data = label_encorder(data)

['s1' 'h2' 'h1' 's2' 'k'],[3 1 0 4 2]


In [10]:
#特徴量の選別 wn_sim_jcnは値がバグっているので使わない(infinityが入っている)
X_data = data.loc[:,feature_columns]
# yの定義
y = data.loc[:,"relation_label"].values.astype("int32")

In [11]:

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

#diffラベルのpca_ssモデル trainとtest arrを返す
def ss_pca(train,pca_n):
    scaler = StandardScaler()
    train=scaler.fit_transform(train)
    pca = PCA(n_components = pca_n)
    train = pca.fit_transform(train)
    return train
#str_arrをarrに変換
def convert_to_arr(vec_str):
    vec_arr = None
    for s in vec_str:
        if vec_arr is None:
            vec_arr =np.array(s[1:-1].split(", ")).astype("float32")
        else:
            vec_arr = np.block([[vec_arr],[np.array(s[1:-1].split(", ")).astype("float32")]])
    return vec_arr

In [14]:
#通常系の特徴量と、vec系の特徴量に分割し、vecにはpcaを行って最後に結合する
X = X_data.loc[:,X_normal_columns].values.astype("float64").astype("float64")
for c_name, n in pca_di.items():
    vec_arr = convert_to_arr(X_data.loc[:,c_name])
    pca_arr = ss_pca(vec_arr,n)
    X = np.concatenate([X, pca_arr], 1)


In [20]:
import optuna

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

#1.目的関数の定義
def objective(trial):
    learning_rate = trial.suggest_uniform('learning_rate', 0.01, 1)
    max_depth = trial.suggest_int('max_depth', 2,20)
    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 10)
    gamma = trial.suggest_uniform('gamma', 0.01, 5)
    subsample = trial.suggest_uniform('subsample', 0.01, 1)
    eta = trial.suggest_uniform("eta",0.1,0.5)
    clf1 = XGBClassifier(learning_rate=learning_rate,
                         max_depth=max_depth,
                         n_estimators=n_estimators,
                         min_child_weight=min_child_weight,
                         gamma=gamma)
    
    score = cross_val_score(clf1, X, y, cv=90)
    print(score)
    return np.mean(score)

#2.Optunaオブジェクトを作成
study = optuna.create_study(direction='maximize')
#3.Optunaでパラメータ探索
study.optimize(objective, n_trials=10)

#4.ベストパラメータ表示
study.best_params


[I 2020-12-14 15:47:03,108] A new study created in memory with name: no-name-6ae60053-dadc-47f2-a7c3-680ae26ffe01
/Users/akihito/.pyenv/versions/3.8.5/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 82 members, which is less than n_splits=90.
  warnings.warn(("The least populated class in y has only %d"


In [ ]:
study = optuna.create_study()
study.optimize(opt, n_trials=100)